# Exeter Eco-Cafe Project

## Introduction

I want to open an eco-friendly cafe in my home city of Exeter in the UK. The city has a wide variety of cafes and eateries, but none of them are climate change thematic. Because of the very large student population in the city, I believe an eco-friendly cafe could be a popular addition to the body of existing cafe franchises.

To begin, I want to consider the cafe's carbon footprint. To tout the cafe as eco-friendly, I will need to demonstrate its comparatively low emissions to its peers. The venue will likely have to be a commercial let. So either I need to find a commercial let with already low building emissions, or I need to find a commercial let with a small footprint so upgrading the infrastructure is cost-effective. If I can demonstrate the venue's low carbon footprint then the business will begin to fulfil its brief as an eco-friendly cafe.

I can use machine learning to cluster the districts of Exeter into three groups by their commercial building emissions, cafe competition, and other relevant metrics. Then I can consider which districts are potentially suitable for my eco-friendly cafe.

## Data
- I will use Energy Performance of Buildings Data for England and Wales to obtain non-domestic Energy Performance Certificates (EPCs) from the Energy Performance of Buildings Register. An EPC contains information about a building's characteristics and CO2 emissions per m2 of floor area. The certificate rates each building A+ to G for its energy efficiency.

    I want access to non-domestic EPCs only, as these represent business/commercial buildings. Domestic EPCs represent homes and are not germane to this investigation.

    I can obtain all of this data from the Ministry of Housing, Communities & Local Government's [Non-Domestic Energy Performance Certificates API](https://epc.opendatacommunities.org/docs/api/non-domestic).

- I will use Foursquare to obtain business venue data by category. I can find the locations and metadata of cafes and other venues within a certain distance of a longitude/latitude coordinate or an address.

    I can obtain all of this data from the Foursquare Developer's [Places API](https://developer.foursquare.com/docs/places-api).

- I will use Lower Layer Super Output Area (LSOA) boundaries to partition Exeter's districts. The boundaries are complied from 2011 census data. They are a geographic hierarchy designed to improve the reporting of small area statistics in England and Wales.

    The LSOA data is published by the Office for National Statistics, but I will obtain the LSOA data from a [GitHub repository](https://martinjc.github.io/UK-GeoJSON) maintained by martinjc. The GitHub repository makes accessing only the relevant geographic data very simple.

- Finally, I need to be able to reference Exeter postcodes to their respective LSOA boundaries, and vice versa. I will use a dataset called Postcode to Output Area to Lower Layer Super Output Area to Middle Layer Super Output Area to Local Authority District (February 2020) Lookup in the UK, also published by the Office for National Statistics.

    I have extracted only the data for the Exeter Local Authority (E07000041) in a CSV file to use in this investigation.

## Data Wrangling

### Libraries

In [3]:
# Data
import numpy as np
import pandas as pd

# Mapping
# !conda install -c conda-forge folium --yes
import folium
# !conda install -c conda-forge geopy --yes
from geopy import Nominatim
import branca
from branca.element import MacroElement
from jinja2 import Template

# Utilities
import re
import requests
from statistics import median, mode

# Clustering
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

print('Libraries imported.')

Libraries imported.


### API Authorisation

In [5]:
# The code was removed by Watson Studio for sharing.

### Functions

In [6]:
def getCoordinate(query, geolocator):
    """Looks up a postcode or address and returns a latitude/longitude tuple.
    
    Parameters:
        query (str): The postcode or address to look up.
        geolocator (obj): A geopy Nominatim object to perform the lookup.
        
    Returns:
        result: A latitude/longitude tuple.
    """
    
    coordinate = geolocator.geocode(query)
    try:
        result = tuple(map(float, [coordinate.raw['lat'], coordinate.raw['lon']]))
    except:
        result = np.nan
    return result

In [7]:
def getNDEPCsByPostcode(postcode,
                        local_authority='E07000041',
                        headers={'Accept':'application/json', **epc_authorization}):
    """Returns JSON containing non-domestic EPCs by postcode within a local authority.
    
        Uses the epc.opendatacommunities.org non-domestic EPC API
        to retrieve non-domestic EPCs by postcode within a local authority.
        Results are returned in JSON format.
    
    Parameters:
        postcode (str): The postcode to look up.
        local_authority (str): The local authority to search within.
        headers (dict): The return format and authorisation headers.
        
    Returns:
        response: JSON containing non-domestic EPCs found within the search parameters.
        
    Raises:
        ConnectionError: If response from the API endpoint is not 200 (OK).
    """
    
    request = 'https://epc.opendatacommunities.org/api/v1/non-domestic/search'
    parameters = f'?postcode={postcode}&local-authority={local_authority}'
    
    url = request + parameters
    
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        raise ConnectionError(f'Response {response.status_code}.')
    else:
        return response.json()

In [8]:
def getVenueSearch(coordinate,
                   categories,
                   radius = 3000,
                   limit=100,
                   version='20210731'):
    """Returns JSON containing venues by category within a certain radius of a latitude/longitude.
    
        Uses the Foursquare Places API
        to retrieve venues by category within a certain radius of a latitude/longitude.
        Results are returned in JSON format.
    
    Parameters:
        coordinate (tuple): The latitude/longitude from which to search.
        categories (list): The venue categories to be returned.
        radius (int): The radius in meters from the coordinate to perform the search.
        limit (int): The response limit.
        version (str): The API version to use.
        
    Returns:
        response: JSON containing venues by category found within the search parameters.
        
    Raises:
        ConnectionError: If response from the API endpoint is not 200 (OK).
    """
    
    coordinate_str = ','.join([str(x) for x in coordinate])
    categories_str = ','.join(categories)
    
    request = 'https://api.foursquare.com/v2/venues/search'
    authorization = f'?client_id={foursquare_client_id}&client_secret={foursquare_client_secret}'
    version = f'&v={version}'
    parameters = f'&ll={coordinate_str}&categoryId={categories_str}&radius={radius}&limit={limit}'
    
    url = request + authorization + version + parameters
    
    response = requests.get(url)
    
    if response.status_code != 200:
        raise ConnectionError(f'Response {response.status_code}.')
    else:
        return response.json()['response']['venues']

In [9]:
"""Returns TopoJSON containing shape data for a local authority.

Uses a GitHub repository maintained by martinjc
to retrieve shape data for a local authority.
Returns TopoJSON.

Parameters:
    local_authority (str): The local authority to lookup.
    level (str): The boundary level data to return.
    
Returns:
    TopoJSON containing shape data for the local authority.
"""

topojson = lambda local_authority, level: requests.get(f'https://martinjc.github.io/UK-GeoJSON/json/eng/{level}_by_lad/topo_{local_authority}.json').json()

In [10]:
# Taken from https://nbviewer.jupyter.org/gist/BibMartin/f153aa957ddc5fadc64929abdee9ff2e
class BindColormap(MacroElement):
    """Binds a colormap to a given layer.

    Parameters
    ----------
    colormap : branca.colormap.ColorMap
        The colormap to bind.
    """
    def __init__(self, layer, colormap):
        super(BindColormap, self).__init__()
        self.layer = layer
        self.colormap = colormap
        self._template = Template(u"""
        {% macro script(this, kwargs) %}
            {{this.colormap.get_name()}}.svg[0][0].style.display = 'block';
            {{this._parent.get_name()}}.on('overlayadd', function (eventLayer) {
                if (eventLayer.layer == {{this.layer.get_name()}}) {
                    {{this.colormap.get_name()}}.svg[0][0].style.display = 'block';
                }});
            {{this._parent.get_name()}}.on('overlayremove', function (eventLayer) {
                if (eventLayer.layer == {{this.layer.get_name()}}) {
                    {{this.colormap.get_name()}}.svg[0][0].style.display = 'none';
                }});
        {% endmacro %}
        """)

### Coordinate
Retrieve Exeter's longitude/latitude.

In [11]:
geolocator = Nominatim(user_agent='my_explorer')
coordinate = getCoordinate('Exeter, United Kingdom', geolocator)
coordinate

(50.7255794, -3.5269497)

### Lookup Table
Load Exeter's postcode to area code lookup table.

In [12]:
# The code was removed by Watson Studio for sharing.

,postcode,oa_code,lsoa_code,msoa_code,la_code
0,EX1 1AA,E00171931,E01020016,E02004156,E07000041
1,EX1 1AB,E00171932,E01020016,E02004156,E07000041
2,EX1 1AD,E00171931,E01020016,E02004156,E07000041
3,EX1 1AE,E00171931,E01020016,E02004156,E07000041
4,EX1 1AF,E00171932,E01020016,E02004156,E07000041


### Venues
I need a DataFrame of cafe or cafe-type venues in Exeter so that I can see what kind of competition I might have.

Categories are taken from Foursquare's [Places API documentation](https://developer.foursquare.com/docs/build-with-foursquare/categories/).

In [13]:
categories = {'bakery':'4bf58dd8d48988d16a941735',
              'cafe':'4bf58dd8d48988d16d941735',
              'coffee_shop':'4bf58dd8d48988d1e0931735'}

Retrieve the JSON containing the venues from the Foursquare [Places API](https://developer.foursquare.com/docs/places-api/).

In [14]:
exeter_venues_json = getVenueSearch(coordinate=coordinate,
                                    categories=categories.values())

I only want venues that have postcodes.  
Filter the JSON to retrieve valid venues only.

In [16]:
# Postcode regex taken from https://stackoverflow.com/questions/23679586/regex-for-uk-postcode
# Credit goes to RobV
pattern = re.compile('[A-Z]{1,2}[0-9]{1,2}[A-Z]?\s?[0-9][A-Z]{2}')

valid_venues = [venue for venue in exeter_venues_json if 
                venue['location'].get('postalCode') and 
                pattern.search(venue['location']['postalCode'])]

valid_venues[0]

{'id': '52b71919498e7539a262bf4b',
 'name': 'Artigiano Espresso Bar',
 'location': {'address': '248 High St',
  'lat': 50.72521043944179,
  'lng': -3.5281112803418457,
  'labeledLatLngs': [{'label': 'display',
    'lat': 50.72521043944179,
    'lng': -3.5281112803418457}],
  'distance': 91,
  'postalCode': 'EX4 3PZ',
  'cc': 'GB',
  'city': 'Exeter',
  'state': 'Devon',
  'country': 'United Kingdom',
  'formattedAddress': ['248 High St',
   'Exeter',
   'Devon',
   'EX4 3PZ',
   'United Kingdom']},
 'categories': [{'id': '4bf58dd8d48988d16d941735',
   'name': 'Café',
   'pluralName': 'Cafés',
   'shortName': 'Café',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_',
    'suffix': '.png'},
   'primary': True}],
 'referralId': 'v-1631375631',
 'hasPerk': False}

Create a DataFrame using the data from the valid venues.

In [17]:
# Columns in this order.
columns = ['id',
           'name',
           'postcode',
           'categories',
           'coordinate']

# DataFrame for valid venues.
venues = pd.DataFrame(columns=columns)

# Iterate through the valid venues.
for venue in valid_venues:
    # Populate the DataFrame with the venue data.
    venues = venues.append({columns[0]:venue['id'],
                            columns[1]:venue['name'],
                            columns[2]:venue['location']['postalCode'],
                            columns[3]:[x['name'] for x in venue['categories']],
                            columns[4]:tuple(map(float, [venue['location']['lat'],
                                                         venue['location']['lng']]))},
                           ignore_index=True)
    
venues['lsoa_code'] = venues['postcode'].map(exeter_pcd_to_oa_lookup.set_index('postcode')['lsoa_code'])

venues

,id,name,postcode,categories,coordinate,lsoa_code
0,52b71919498e7539a262bf4b,Artigiano Espresso Bar,EX4 3PZ,[Café],"(50.72521043944179, -3.5281112803418457)",E01020016
1,4b533692f964a520889227e3,Starbucks,EX4 3LF,[Coffee Shop],"(50.725003, -3.528024)",E01020016
2,4f16b1d7e4b0ac7374106bef,Coffee #1,EX1 1GN,[Coffee Shop],"(50.723934788136646, -3.527016812594657)",E01020016
3,601ea846a1c89144b3bd34e9,crankhouse coffee,EX4 3JQ,[Coffee Shop],"(50.721998, -3.533819)",E01020016
4,58ab428d5cab2f6649f6c7c8,Puerto Lounge,EX2 4AE,[Café],"(50.718438, -3.532146)",E01020016
5,52f273c7498eb3bd7af0fbc5,Patisserie Valerie,EX4 3DU,[Café],"(50.72301845183361, -3.5321274284828124)",E01020018
6,5738db5ecd10aa5138b269a1,Starbucks,EX2 8JD,[Coffee Shop],"(50.707949, -3.5371)",E01019972
7,4e57a0b01fc7d639d0d2e34e,Cafe Espresso,EX4 3PT,[Café],"(50.725485116386736, -3.5293834472469388)",E01020016
8,4c066b785753c9283ad83af1,Costa Coffee,EX1 1LL,[Coffee Shop],"(50.72373054265367, -3.528447185596214)",E01020016
9,5790af08cd10f3265885202e,Chococo,EX4 3LS,[Café],"(50.724914, -3.531498)",E01020018


### EPCs
I need a DataFrame of all non-domestic EPCs by postcode so that I can create a building emissions profile of Exeter.  
Create a DataFrame using the data from the epc.opendatacommunities.org [non-domestic EPC API](https://epc.opendatacommunities.org/docs/api/non-domestic).

In [18]:
# Columns in this order.
columns = ['postcode',
           'lsoa_code',
           'energy_ratings',
           'building_levels',
           'main_heating_fuels',
           'floor_areas',
           'building_emissions']

# DataFrame for EPC data by postcode.
epc_data_by_pcd = pd.DataFrame(columns=columns)

# Iterate through each postcode in the lookup table.
for index, row in exeter_pcd_to_oa_lookup.iterrows():
    # Get non-domestic EPCs for each postcode.
    try:
        non_domestic_epcs_json = getNDEPCsByPostcode(postcode=row['postcode'].strip()) # API takes postcodes without whitespace.
    except:
        continue # Skip row if EPC API can't retrieve for the postcode.
    
    # Populate the DataFrame with the EPC data if data has been retrieved.
    epc_data_by_pcd = epc_data_by_pcd.append({columns[0]:row['postcode'],
                                              columns[1]:exeter_pcd_to_oa_lookup[exeter_pcd_to_oa_lookup['postcode'] == row['postcode']]['lsoa_code'].item(),
                                              columns[2]:[x['asset-rating-band'] for x in non_domestic_epcs_json['rows'] if ['asset-rating-band']],
                                              columns[3]:[x['building-level'] for x in non_domestic_epcs_json['rows'] if x['building-level']],
                                              columns[4]:[x['main-heating-fuel'] for x in non_domestic_epcs_json['rows'] if x['main-heating-fuel']],
                                              columns[5]:[x['floor-area'] for x in non_domestic_epcs_json['rows'] if x['floor-area']],
                                              columns[6]:[x['building-emissions'] for x in non_domestic_epcs_json['rows'] if x['building-emissions']]},
                                             ignore_index=True)
        
epc_data_by_pcd.head()

,postcode,lsoa_code,energy_ratings,building_levels,main_heating_fuels,floor_areas,building_emissions
0,EX1 1AF,E01020016,"[C, C]","[3, 3]","[Natural Gas, Natural Gas]","[261, 264]","[62.78, 61.7]"
1,EX1 1AL,E01020016,[B],[5],[Grid Supplied Electricity],[711],[26.27]
2,EX1 1AP,E01020016,"[C, C]","[3, 3]","[Natural Gas, Natural Gas]","[978, 338]","[39.25, 58.34]"
3,EX1 1AR,E01020016,"[C, F]","[3, 3]","[Grid Supplied Electricity, Grid Supplied Elec...","[198, 195]","[95.08, 228.59]"
4,EX1 1BA,E01020016,"[F, E]","[3, 3]","[Natural Gas, Grid Supplied Electricity]","[60, 50]",[187.64]


### LSOA Codes
Create a DataFrame of Exeter's unique LSOA codes.

In [19]:
lsoa_codes = pd.DataFrame({'lsoa_code':exeter_pcd_to_oa_lookup['lsoa_code'].unique()})
lsoa_codes.head()

,lsoa_code
0,E01020016
1,E01019995
2,E01019996
3,E01019987
4,E01019994


Count how many venues are in each LSOA.

In [20]:
venues_by_lsoa = lsoa_codes.set_index('lsoa_code')
venues_by_lsoa['venue_count'] = venues.groupby('lsoa_code', dropna=True)['id'].agg(len)
venues_by_lsoa = venues_by_lsoa.replace(np.nan, 0)
venues_by_lsoa['venue_count'] = venues_by_lsoa['venue_count'].astype('int64')
venues_by_lsoa.head()

,venue_count
lsoa_code,
E01020016,15
E01019995,0
E01019996,1
E01019987,0
E01019994,0


## Methodology
I now have all of the data I need to start analysing Exeter's building emissions profile.  
I will create some choropleth maps of the data and then perform some clustering.

### EPCs by LSOA
I need to group the EPCs by LSOA so that I can create some choropleths of the data.  
I can also engineer some new features from the data to use in mapping and in clustering.  
Create a new DataFrame using the EPC by postcode data.

In [21]:
epc_data_by_lsoa = epc_data_by_pcd.groupby('lsoa_code', as_index=False).agg({'energy_ratings':'sum',
                                                                             'main_heating_fuels':'sum',
                                                                             'floor_areas':'sum',
                                                                             'building_emissions':'sum'})

epc_data_by_lsoa['epc_count'] = epc_data_by_lsoa['energy_ratings'].apply(len, convert_dtype=False)
epc_data_by_lsoa['epc_mode'] = epc_data_by_lsoa['energy_ratings'].apply(mode)
epc_data_by_lsoa['rating_A+'] = epc_data_by_lsoa['energy_ratings'].apply(lambda x: len([i for i in x if i == 'A+']), convert_dtype=False)
epc_data_by_lsoa['rating_A'] = epc_data_by_lsoa['energy_ratings'].apply(lambda x: len([i for i in x if i == 'A']), convert_dtype=False)
epc_data_by_lsoa['rating_B'] = epc_data_by_lsoa['energy_ratings'].apply(lambda x: len([i for i in x if i == 'B']), convert_dtype=False)
epc_data_by_lsoa['rating_C'] = epc_data_by_lsoa['energy_ratings'].apply(lambda x: len([i for i in x if i == 'C']), convert_dtype=False)
epc_data_by_lsoa['rating_D'] = epc_data_by_lsoa['energy_ratings'].apply(lambda x: len([i for i in x if i == 'D']), convert_dtype=False)
epc_data_by_lsoa['rating_E'] = epc_data_by_lsoa['energy_ratings'].apply(lambda x: len([i for i in x if i == 'E']), convert_dtype=False)
epc_data_by_lsoa['rating_F'] = epc_data_by_lsoa['energy_ratings'].apply(lambda x: len([i for i in x if i == 'F']), convert_dtype=False)
epc_data_by_lsoa['rating_G'] = epc_data_by_lsoa['energy_ratings'].apply(lambda x: len([i for i in x if i == 'G']), convert_dtype=False)
epc_data_by_lsoa['fuels_mode'] = epc_data_by_lsoa['main_heating_fuels'].apply(mode)
epc_data_by_lsoa['gas_count'] = epc_data_by_lsoa['main_heating_fuels'].apply(lambda x: len([i for i in x if i == 'Natural Gas']), convert_dtype=False)
epc_data_by_lsoa['electricity_count'] = epc_data_by_lsoa['main_heating_fuels'].apply(lambda x: len([i for i in x if i == 'Grid Supplied Electricity']), convert_dtype=False)
epc_data_by_lsoa['floor_areas_median'] = epc_data_by_lsoa['floor_areas'].apply(lambda x: median([float(i) for i in x]))
epc_data_by_lsoa['emissions_median'] = epc_data_by_lsoa['building_emissions'].apply(lambda x: median([float(i) for i in x]) if x else 0)

epc_data_by_lsoa = lsoa_codes.set_index('lsoa_code').join(epc_data_by_lsoa.set_index('lsoa_code'))

epc_data_by_lsoa = epc_data_by_lsoa.replace(np.nan, 0)
epc_data_by_lsoa = epc_data_by_lsoa.drop(columns=['energy_ratings',
                                                  'main_heating_fuels',
                                                  'floor_areas',
                                                  'building_emissions'],
                                         axis=1)

epc_data_by_lsoa.head()

,epc_count,epc_mode,rating_A+,rating_A,rating_B,rating_C,rating_D,rating_E,rating_F,rating_G,fuels_mode,gas_count,electricity_count,floor_areas_median,emissions_median
lsoa_code,,,,,,,,,,,,,,,
E01020016,547,D,0,4,25,135,168,95,38,82,Grid Supplied Electricity,200,340,217.0,81.960
E01019995,130,D,0,1,12,28,40,22,11,16,Grid Supplied Electricity,50,78,139.0,99.650
E01019996,43,C,0,0,3,14,13,7,4,2,Natural Gas,24,17,173.0,94.865
E01019987,53,C,0,0,7,17,13,7,4,5,Grid Supplied Electricity,24,29,111.0,102.140
E01019994,20,D,0,1,2,6,6,2,0,3,Natural Gas,12,8,103.0,87.965


### Choropleth Maps
Create choropleth maps for some of the features in the EPC by LSOA DataFrame.

In [23]:
exeter_map = folium.Map(location=coordinate,
                        tiles=None,
                        zoom_start=13)

folium.TileLayer(tiles='cartodbpositron',
                 min_zoom=13,
                 control=False).add_to(exeter_map)

exeter_topojson = topojson('E07000041', 'lsoa')

highlight_function = lambda x: {'fillColor':'#000000',
                                'color':'#000000',
                                'fillOpacity':0.5,
                                'weight':0.1}

# Venue markers layer
venue_markers = folium.FeatureGroup(name='Venue markers',
                                    overlay=True)

venues.apply(lambda venue: folium.CircleMarker(location=[venue['coordinate'][0], venue['coordinate'][1]],
                                               tooltip=venue['name'],
                                               fill=True,
                                               stroke=False,
                                               color='black',
                                               fill_opacity=1.0,
                                               radius=4).add_to(venue_markers), axis=1)

exeter_map.add_child(venue_markers)
exeter_map.keep_in_front(venue_markers)

# Venue count layer
venue_count_colourmap = branca.colormap.LinearColormap(colors=branca.colormap.linear.GnBu_09.colors,
                                                       vmin=venues_by_lsoa['venue_count'].min(),
                                                       vmax=venues_by_lsoa['venue_count'].max(),
                                                       caption='Number of Venues')

venue_count_style_function = lambda x: {'weight':0.1,
                                        'color':'black',
                                        'fillColor':venue_count_colourmap(venues_by_lsoa['venue_count'].get(x['id'])),
                                        'fillOpacity':0.3}

venue_count_topojson = folium.features.TopoJson(data=exeter_topojson,
                                                object_path='objects.E07000041',
                                                style_function=venue_count_style_function,
                                                name='Venue count',
                                                overlay=True,
                                                show=False)

venue_count_bind = BindColormap(venue_count_topojson, venue_count_colourmap)
exeter_map.add_child(venue_count_colourmap)
exeter_map.add_child(venue_count_topojson)
exeter_map.add_child(venue_count_bind)

# Emissions layer
emissions_median_colourmap = branca.colormap.LinearColormap(colors=branca.colormap.linear.OrRd_09.colors,
                                                            vmin=epc_data_by_lsoa['emissions_median'].min(),
                                                            vmax=epc_data_by_lsoa['emissions_median'].max(),
                                                            caption='Median Building Emissions (kg CO2/m2)')

emissions_median_style_function = lambda x: {'weight':0.1,
                                             'color':'black',
                                             'fillColor':emissions_median_colourmap(epc_data_by_lsoa['emissions_median'].get(x['id'])),
                                             'fillOpacity':0.3}

emissions_median_topojson = folium.features.TopoJson(data=exeter_topojson,
                                                     object_path='objects.E07000041',
                                                     style_function=emissions_median_style_function,
                                                     name='Median emissions',
                                                     overlay=True,
                                                     show=True)

emissions_median_bind = BindColormap(emissions_median_topojson, emissions_median_colourmap)
exeter_map.add_child(emissions_median_colourmap)
exeter_map.add_child(emissions_median_topojson)
exeter_map.add_child(emissions_median_bind)

# EPC count layer
epc_count_colourmap = branca.colormap.LinearColormap(colors=branca.colormap.linear.GnBu_09.colors,
                                                     vmin=epc_data_by_lsoa['epc_count'].min(),
                                                     vmax=epc_data_by_lsoa['epc_count'].max(),
                                                     caption='Number of Non-Domestic EPCs Recorded')

epc_count_style_function = lambda x: {'weight':0.1,
                                      'color':'black',
                                      'fillColor':epc_count_colourmap(epc_data_by_lsoa['epc_count'].get(x['id'])),
                                      'fillOpacity':0.3}

epc_count_topojson = folium.features.TopoJson(data=exeter_topojson,
                                              object_path='objects.E07000041',
                                              style_function=epc_count_style_function,
                                              name='EPC count',
                                              overlay=True,
                                              show=False)

epc_count_bind = BindColormap(epc_count_topojson, epc_count_colourmap)
exeter_map.add_child(epc_count_colourmap)
exeter_map.add_child(epc_count_topojson)
exeter_map.add_child(epc_count_bind)

# Gas count layer
gas_count_colourmap = branca.colormap.LinearColormap(colors=branca.colormap.linear.GnBu_09.colors,
                                                     vmin=epc_data_by_lsoa['gas_count'].min(),
                                                     vmax=epc_data_by_lsoa['gas_count'].max(),
                                                     caption='Number of Non-Domestic Venues Supplied by Natural Gas')

gas_count_style_function = lambda x: {'weight':0.1,
                                      'color':'black',
                                      'fillColor':gas_count_colourmap(epc_data_by_lsoa['gas_count'].get(x['id'])),
                                      'fillOpacity':0.3}

gas_count_topojson = folium.features.TopoJson(data=exeter_topojson,
                                              object_path='objects.E07000041',
                                              style_function=gas_count_style_function,
                                              name='Gas count',
                                              overlay=True,
                                              show=False)

gas_count_bind = BindColormap(gas_count_topojson, gas_count_colourmap)
exeter_map.add_child(gas_count_colourmap)
exeter_map.add_child(gas_count_topojson)
exeter_map.add_child(gas_count_bind)

# Electricity count layer
electricity_count_colourmap = branca.colormap.LinearColormap(colors=branca.colormap.linear.GnBu_09.colors,
                                                             vmin=epc_data_by_lsoa['electricity_count'].min(),
                                                             vmax=epc_data_by_lsoa['electricity_count'].max(),
                                                             caption='Number of Non-Domestic Venues Supplied by Electricity')

electricity_count_style_function = lambda x: {'weight':0.1,
                                              'color':'black',
                                              'fillColor':electricity_count_colourmap(epc_data_by_lsoa['electricity_count'].get(x['id'])),
                                              'fillOpacity':0.3}

electricity_count_topojson = folium.features.TopoJson(data=exeter_topojson,
                                                      object_path='objects.E07000041',
                                                      style_function=electricity_count_style_function,
                                                      name='Electricity count',
                                                      overlay=True,
                                                      show=False)

electricity_count_bind = BindColormap(electricity_count_topojson, electricity_count_colourmap)
exeter_map.add_child(electricity_count_colourmap)
exeter_map.add_child(electricity_count_topojson)
exeter_map.add_child(electricity_count_bind)

# Floor areas layer
floor_areas_median_colourmap = branca.colormap.LinearColormap(colors=branca.colormap.linear.GnBu_09.colors,
                                                              vmin=epc_data_by_lsoa['floor_areas_median'].min(),
                                                              vmax=epc_data_by_lsoa['floor_areas_median'].max(),
                                                              caption='Median Floor Areas (m2)')

floor_areas_median_style_function = lambda x: {'weight':0.1,
                                               'color':'black',
                                               'fillColor':floor_areas_median_colourmap(epc_data_by_lsoa['floor_areas_median'].get(x['id'])),
                                               'fillOpacity':0.3}

floor_areas_median_topojson = folium.features.TopoJson(data=exeter_topojson,
                                                       object_path='objects.E07000041',
                                                       style_function=floor_areas_median_style_function,
                                                       name='Median fLoor areas',
                                                       overlay=True,
                                                       show=False)

floor_areas_median_bind = BindColormap(floor_areas_median_topojson, floor_areas_median_colourmap)
exeter_map.add_child(floor_areas_median_colourmap)
exeter_map.add_child(floor_areas_median_topojson)
exeter_map.add_child(floor_areas_median_bind)

# A+ layer
a_plus_colourmap = branca.colormap.LinearColormap(colors=branca.colormap.linear.YlGn_09.colors,
                                                  index=np.linspace(start=epc_data_by_lsoa['rating_A+'].min(),
                                                                    stop=epc_data_by_lsoa['rating_A+'].max(),
                                                                    num=3),
                                                  vmin=epc_data_by_lsoa['rating_A+'].min(),
                                                  vmax=epc_data_by_lsoa['rating_A+'].max(),
                                                  caption='Number of A+ Energy Ratings Recorded')

a_plus_style_function = lambda x: {'weight':0.1,
                                   'color':'black',
                                   'fillColor':a_plus_colourmap(epc_data_by_lsoa['rating_A+'].get(x['id'])),
                                   'fillOpacity':0.3}

a_plus_topojson = folium.features.TopoJson(data=exeter_topojson,
                                           object_path='objects.E07000041',
                                           style_function=a_plus_style_function,
                                           name='A+ ratings',
                                           overlay=True,
                                           show=False)

a_plus_bind = BindColormap(a_plus_topojson, a_plus_colourmap)
exeter_map.add_child(a_plus_colourmap)
exeter_map.add_child(a_plus_topojson)
exeter_map.add_child(a_plus_bind)

# A layer
a_colourmap = branca.colormap.LinearColormap(colors=branca.colormap.linear.YlGn_09.colors,
                                             index=np.linspace(start=epc_data_by_lsoa['rating_A'].min(),
                                                               stop=epc_data_by_lsoa['rating_A'].max(),
                                                               num=5),
                                             vmin=epc_data_by_lsoa['rating_A'].min(),
                                             vmax=epc_data_by_lsoa['rating_A'].max(),
                                             caption='Number of A Energy Ratings Recorded')

a_style_function = lambda x: {'weight':0.1,
                              'color':'black',
                              'fillColor':a_colourmap(epc_data_by_lsoa['rating_A'].get(x['id'])),
                              'fillOpacity':0.3}

a_topojson = folium.features.TopoJson(data=exeter_topojson,
                                      object_path='objects.E07000041',
                                      style_function=a_style_function,
                                      name='A ratings',
                                      overlay=True,
                                      show=False)

a_bind = BindColormap(a_topojson, a_colourmap)
exeter_map.add_child(a_colourmap)
exeter_map.add_child(a_topojson)
exeter_map.add_child(a_bind)

# B layer
b_colourmap = branca.colormap.LinearColormap(colors=branca.colormap.linear.YlGn_09.colors,
                                             index=np.linspace(start=epc_data_by_lsoa['rating_B'].min(),
                                                               stop=epc_data_by_lsoa['rating_B'].max(),
                                                               num=9),
                                             vmin=epc_data_by_lsoa['rating_B'].min(),
                                             vmax=epc_data_by_lsoa['rating_B'].max(),
                                             caption='Number of B Energy Ratings Recorded')

b_style_function = lambda x: {'weight':0.1,
                              'color':'black',
                              'fillColor':b_colourmap(epc_data_by_lsoa['rating_B'].get(x['id'])),
                              'fillOpacity':0.3}

b_topojson = folium.features.TopoJson(data=exeter_topojson,
                                      object_path='objects.E07000041',
                                      style_function=b_style_function,
                                      name='B ratings',
                                      overlay=True,
                                      show=False)

b_bind = BindColormap(b_topojson, b_colourmap)
exeter_map.add_child(b_colourmap)
exeter_map.add_child(b_topojson)
exeter_map.add_child(b_bind)

# C layer
c_colourmap = branca.colormap.LinearColormap(colors=branca.colormap.linear.YlGn_09.colors,
                                             index=np.linspace(start=epc_data_by_lsoa['rating_C'].min(),
                                                               stop=epc_data_by_lsoa['rating_C'].max(),
                                                               num=8),
                                             vmin=epc_data_by_lsoa['rating_C'].min(),
                                             vmax=epc_data_by_lsoa['rating_C'].max(),
                                             caption='Number of C Energy Ratings Recorded')

c_style_function = lambda x: {'weight':0.1,
                              'color':'black',
                              'fillColor':c_colourmap(epc_data_by_lsoa['rating_C'].get(x['id'])),
                              'fillOpacity':0.3}

c_topojson = folium.features.TopoJson(data=exeter_topojson,
                                      object_path='objects.E07000041',
                                      style_function=c_style_function,
                                      name='C ratings',
                                      overlay=True,
                                      show=False)

c_bind = BindColormap(c_topojson, c_colourmap)
exeter_map.add_child(c_colourmap)
exeter_map.add_child(c_topojson)
exeter_map.add_child(c_bind)

# D layer
d_colourmap = branca.colormap.LinearColormap(colors=branca.colormap.linear.OrRd_09.colors,
                                             index=np.linspace(start=epc_data_by_lsoa['rating_D'].min(),
                                                               stop=epc_data_by_lsoa['rating_D'].max(),
                                                               num=9),
                                             vmin=epc_data_by_lsoa['rating_D'].min(),
                                             vmax=epc_data_by_lsoa['rating_D'].max(),
                                             caption='Number of D Energy Ratings Recorded')

d_style_function = lambda x: {'weight':0.1,
                              'color':'black',
                              'fillColor':d_colourmap(epc_data_by_lsoa['rating_D'].get(x['id'])),
                              'fillOpacity':0.3}

d_topojson = folium.features.TopoJson(data=exeter_topojson,
                                      object_path='objects.E07000041',
                                      style_function=d_style_function,
                                      name='D ratings',
                                      overlay=True,
                                      show=False)

d_bind = BindColormap(d_topojson, d_colourmap)
exeter_map.add_child(d_colourmap)
exeter_map.add_child(d_topojson)
exeter_map.add_child(d_bind)

# E layer
e_colourmap = branca.colormap.LinearColormap(colors=branca.colormap.linear.OrRd_09.colors,
                                             index=np.linspace(start=epc_data_by_lsoa['rating_E'].min(),
                                                               stop=epc_data_by_lsoa['rating_E'].max(),
                                                               num=9),
                                             vmin=epc_data_by_lsoa['rating_E'].min(),
                                             vmax=epc_data_by_lsoa['rating_E'].max(),
                                             caption='Number of E Energy Ratings Recorded')

e_style_function = lambda x: {'weight':0.1,
                              'color':'black',
                              'fillColor':e_colourmap(epc_data_by_lsoa['rating_E'].get(x['id'])),
                              'fillOpacity':0.3}

e_topojson = folium.features.TopoJson(data=exeter_topojson,
                                      object_path='objects.E07000041',
                                      style_function=e_style_function,
                                      name='E ratings',
                                      overlay=True,
                                      show=False)

e_bind = BindColormap(e_topojson, e_colourmap)
exeter_map.add_child(e_colourmap)
exeter_map.add_child(e_topojson)
exeter_map.add_child(e_bind)

# F layer
f_colourmap = branca.colormap.LinearColormap(colors=branca.colormap.linear.OrRd_09.colors,
                                             index=np.linspace(start=epc_data_by_lsoa['rating_F'].min(),
                                                               stop=epc_data_by_lsoa['rating_F'].max(),
                                                               num=9),
                                             vmin=epc_data_by_lsoa['rating_F'].min(),
                                             vmax=epc_data_by_lsoa['rating_F'].max(),
                                             caption='Number of F Energy Ratings Recorded')

f_style_function = lambda x: {'weight':0.1,
                              'color':'black',
                              'fillColor':f_colourmap(epc_data_by_lsoa['rating_F'].get(x['id'])),
                              'fillOpacity':0.3}

f_topojson = folium.features.TopoJson(data=exeter_topojson,
                                      object_path='objects.E07000041',
                                      style_function=f_style_function,
                                      name='F ratings',
                                      overlay=True,
                                      show=False)

f_bind = BindColormap(f_topojson, f_colourmap)
exeter_map.add_child(f_colourmap)
exeter_map.add_child(f_topojson)
exeter_map.add_child(f_bind)

# G layer
g_colourmap = branca.colormap.LinearColormap(colors=branca.colormap.linear.OrRd_09.colors,
                                             index=np.linspace(start=epc_data_by_lsoa['rating_G'].min(),
                                                               stop=epc_data_by_lsoa['rating_G'].max(),
                                                               num=9),
                                             vmin=epc_data_by_lsoa['rating_G'].min(),
                                             vmax=epc_data_by_lsoa['rating_G'].max(),
                                             caption='Number of G Energy Ratings Recorded')

g_style_function = lambda x: {'weight':0.1,
                              'color':'black',
                              'fillColor':g_colourmap(epc_data_by_lsoa['rating_G'].get(x['id'])),
                              'fillOpacity':0.3}

g_topojson = folium.features.TopoJson(data=exeter_topojson,
                                      object_path='objects.E07000041',
                                      style_function=g_style_function,
                                      name='G ratings',
                                      overlay=True,
                                      show=False)

g_bind = BindColormap(g_topojson, g_colourmap)
exeter_map.add_child(g_colourmap)
exeter_map.add_child(g_topojson)
exeter_map.add_child(g_bind)

layer_control = folium.map.LayerControl(position='topleft',
                                        collapsed=False,
                                        autoZIndex=True).add_to(exeter_map)

exeter_map

### K-Means Clustering
I need to create a DataFrame to perform clustering on.  
Create a new DataFrame from the EPC by LSOA and the venues by LSOA DataFrames.  
Drop some of the features to make the clustering process more coherent.

In [24]:
exeter_by_lsoa = venues_by_lsoa.join(epc_data_by_lsoa)
exeter_by_lsoa = pd.get_dummies(data=exeter_by_lsoa,
                                drop_first=True,
                                prefix='mode_',
                                prefix_sep='')

columns_to_drop = ['epc_count',
                   'rating_A+',
                   'rating_A',
                   'rating_B',
                   'rating_C',
                   'rating_D',
                   'rating_E',
                   'rating_F',
                   'rating_G',
                   'gas_count',
                   'electricity_count']

exeter_by_lsoa.drop(columns_to_drop, axis=1, inplace=True)

exeter_by_lsoa.head()

,venue_count,floor_areas_median,emissions_median,mode_B,mode_C,mode_D,mode_E,mode_Grid Supplied Electricity,mode_Natural Gas
lsoa_code,,,,,,,,,
E01020016,15,217.0,81.960,0,0,1,0,1,0
E01019995,0,139.0,99.650,0,0,1,0,1,0
E01019996,1,173.0,94.865,0,1,0,0,0,1
E01019987,0,111.0,102.140,0,1,0,0,1,0
E01019994,0,103.0,87.965,0,0,1,0,0,1


Scale the features.

In [25]:
X = exeter_by_lsoa.values
exeter_by_lsoa_scaled = StandardScaler().fit_transform(X)

Perform clustering and assign each LSOA a label.

In [26]:
k = 3

k_means = KMeans(n_clusters=k,
                 init='k-means++',
                 n_init=12,
                 max_iter=300,
                 random_state=1).fit(exeter_by_lsoa_scaled)
exeter_by_lsoa.insert(0, 'k_means_label', k_means.labels_)

exeter_by_lsoa.head()

,k_means_label,venue_count,floor_areas_median,emissions_median,mode_B,mode_C,mode_D,mode_E,mode_Grid Supplied Electricity,mode_Natural Gas
lsoa_code,,,,,,,,,,
E01020016,0,15,217.0,81.960,0,0,1,0,1,0
E01019995,0,0,139.0,99.650,0,0,1,0,1,0
E01019996,2,1,173.0,94.865,0,1,0,0,0,1
E01019987,0,0,111.0,102.140,0,1,0,0,1,0
E01019994,2,0,103.0,87.965,0,0,1,0,0,1


## Results
Each LSOA now belongs to a cluster.  
To visualise the results, I will map the clusters to a choropleth.

In [27]:
cluster_map = folium.Map(location=coordinate,
                         tiles=None,
                         zoom_start=13)

folium.TileLayer(tiles='cartodbpositron',
                 min_zoom=13,
                 control=False).add_to(cluster_map)

# Taken from https://towardsdatascience.com/clustering-geospatial-data-f0584f0b04ec
clusters = sorted(list(exeter_by_lsoa['k_means_label'].unique()))
colours = ['#%06X' % np.random.randint(0, 0xFFFFFF) for i in range(len(clusters))]
exeter_by_lsoa['colour'] = exeter_by_lsoa['k_means_label'].apply(lambda x: colours[clusters.index(x)])


cluster_style_function = lambda x: {'weight':0.1,
                                    'color':'black',
                                    'fillColor':exeter_by_lsoa['colour'].get(x['id']),
                                    'fillOpacity':0.3}

#taken from https://vverde.github.io/blob/interactivechoropleth.html
tooltip = folium.features.GeoJsonTooltip(fields=['LSOA11CD'],
                                         aliases=['LSOA Code'],
                                         style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"),
                                         sticky=True)

cluster_topojson = folium.features.TopoJson(data=exeter_topojson,
                                            object_path='objects.E07000041',
                                            style_function=cluster_style_function,
                                            name='Clusters',
                                            tooltip=tooltip).add_to(cluster_map)

# Taken from https://towardsdatascience.com/clustering-geospatial-data-f0584f0b04ec
legend_html = """<div style="position:fixed; top:10px; right:10px; border:1px solid black; padding:10px; z-index:9999; font-size:12px;">&nbsp;<b>Clusters:</b><br>"""
for i in clusters:
     legend_html = legend_html + """&nbsp;<i class="fa fa-circle 
     fa-1x" style="color:""" + colours[clusters.index(i)] + """">
     </i>&nbsp;""" + str(i) + """<br>"""
legend_html = legend_html + """</div>"""
cluster_map.get_root().html.add_child(folium.Element(legend_html))

cluster_map

I want to profile each cluster.  
Create a new DataFrame of aggregated features by cluster label.

In [28]:
clusters_average_statistics = exeter_by_lsoa.groupby('k_means_label').agg({'venue_count':'sum',
                                                                           'floor_areas_median':'mean',
                                                                           'emissions_median':'mean',
                                                                           'mode_B':'sum',
                                                                           'mode_C':'sum',
                                                                           'mode_D':'sum',
                                                                           'mode_E':'sum',
                                                                           'mode_Grid Supplied Electricity':'sum',
                                                                           'mode_Natural Gas':'sum'})

clusters_average_statistics['lsoa_count'] = exeter_by_lsoa.groupby('k_means_label')['venue_count'].count()
clusters_average_statistics.T

k_means_label,0,1,2
venue_count,25.000000,0.000000,2.000000
floor_areas_median,200.533333,681.916667,320.026316
emissions_median,97.038333,52.980833,64.421184
mode_B,0.000000,6.000000,0.000000
mode_C,11.000000,0.000000,17.000000
mode_D,16.000000,0.000000,10.000000
mode_E,3.000000,0.000000,4.000000
mode_Grid Supplied Electricity,30.000000,0.000000,0.000000
mode_Natural Gas,0.000000,6.000000,31.000000
lsoa_count,30.000000,6.000000,38.000000


## Discussion
I can profile each cluster by observing its characteristics.

### Cluster 0  
✅ High competition  
✅ Small footprint  
✅ High emissions  
✅ Low ratings  
✅ Electricity supplied  
### Cluster 1  
✅ Low competition  
✅ Medium footprint  
✅ Medium emissions  
✅ Medium ratings  
✅ Gas supplied  
### Cluster 2  
✅ Low competition  
✅ Large footptint  
✅ Low emissions  
✅ High ratings  
✅ Gas supplied  

Opening an eco-cafe in Exeter will present different challenges depending on which of these zones the venue might be located.  

Cluster 0 will put the venue in a high competition area, but the unique theme of an eco-cafe is niche. Most of the non-domestic buildings in this zone have small m2 footprints, with high emissions and low emissions ratings. That means my eco-cafe would need to invest in energy-saving infrastructure. But the low footprint of the buildings could mean that any infrastructure investments are not too high.

Cluster 1 will put the venue in a low competition area. Most of the non-domestic buildings in this zone have medium emissions ratings and medium m2 footprints. Upgrading the infrastructure in these buildings may be more costly than in cluster 0, depending on the type of investment required. Most of the buildings are gas supplied, which may be more conducive to a cafe.

Cluster 2 will also put the venue in low competition areas, but these are far away from the town centre. Although the non-domestic buildings in this tend to have low emissions and high emission ratings, they have large m2 footprints and are often specialised. Suitable venues may be sparse.

## Conclusion
I want to open an eco-friendly cafe in my home city of Exeter in the UK.

In this project I used a K-Means clustering machine learning algorithm to partition the Lower Layer Super Output Areas of Exeter into three zones by the energy emissions ratings of their non-domestic buildings and by the competition posed by other cafe venues.

I can conclude that the three zones pose very different pros and cons for the viability of my eco-friendly cafe idea. Exeter appears to have a wide cohort of non-domestic buildings to choose from, and competition appears to be geographically concentrated. I have lots of angles from which to market my venue's niche concept.

The final decision for the cafe's locale will be made in conjunction with my investors and other experts, with the support of the data provided in this study.